Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [2]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


### Define Pipeline Components

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [133]:
vect = TfidfVectorizer(stop_words='english', 
                       ngram_range=(1,2),
                       max_features=10000)
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [153]:
parameters = {
    'vect__max_df': (0.8, 0.97),
    'vect__min_df': (.01, .02),
    'vect__max_features': (1000,1500,2000),
    'clf__n_estimators':(15, 20, 25),
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.1min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=10000,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        n

In [155]:
grid_search.best_params_

{'clf__n_estimators': 25,
 'vect__max_df': 0.97,
 'vect__max_features': 1500,
 'vect__min_df': 0.01}

In [154]:
grid_search.best_score_

0.7313503095887214

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [148]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [149]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [150]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [89]:
subNumber = 3

In [151]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [109]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100,
                   algorithm='randomized',
                   n_iter=10)

vect = TfidfVectorizer(stop_words='english',
                       ngram_range=(1,2))

clf = RandomForestClassifier()

In [110]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'vect__max_df': (0.8, 0.9, 1.0),
    'clf__max_depth':(5,10,15,20)
}

In [111]:
lsi = Pipeline([('vect', vect), ('svd', svd)])


pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [112]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': (0.75, 0.8, 0.85),
    'lsi__vect__min_df': (0.01, 0.02),
    'lsi__vect__max_features': (1250, 1500, 1750),
    'clf__n_estimators':(30, 35, 40),
    'clf__max_depth':(25, 30, 35)
}


grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 486 candidates, totalling 2430 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   56.9s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 2430 out of 2430 | elapsed: 18.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [113]:
grid_search.best_score_

0.7279213414214446

In [114]:
grid_search.best_params_

{'clf__max_depth': 25,
 'clf__n_estimators': 35,
 'lsi__svd__n_components': 100,
 'lsi__vect__max_df': 0.85,
 'lsi__vect__max_features': 1500,
 'lsi__vect__min_df': 0.01}

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [ ]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [116]:
# Continue Word Embedding Work Here
import spacy

nlp = spacy.load('en_core_web_lg')

In [117]:
def get_word_vectors(docs):
    return[nlp(doc).vector for doc in docs]

In [118]:
X = get_word_vectors(train['description'])

In [119]:
X[0]

array([-4.67550606e-02,  2.17486411e-01, -1.39248788e-01, -8.63811150e-02,
        1.08062603e-01,  8.21812302e-02, -6.99262992e-02, -7.88967833e-02,
       -4.63048331e-02,  1.67534256e+00, -8.29739422e-02,  8.47098455e-02,
       -1.61168594e-02, -1.15673296e-01, -1.00931458e-01, -1.48674801e-01,
       -2.86327489e-02,  1.00717258e+00, -8.84611458e-02, -3.00116055e-02,
       -2.12408844e-02, -1.98691525e-03, -2.18204875e-02, -2.43686680e-02,
       -8.50067884e-02, -5.28534129e-02, -3.88755389e-02,  2.85331737e-02,
        9.71467644e-02, -1.52220830e-01, -7.67232990e-03,  3.68110649e-02,
        3.03680412e-02, -3.20375822e-02,  4.51759845e-02, -2.22320929e-02,
        2.99556088e-02, -3.53264883e-02, -8.09046626e-02,  4.02572937e-02,
       -4.52380776e-02,  4.00653407e-02,  1.01217575e-01, -1.02363355e-01,
        6.43703416e-02,  1.21800080e-01, -8.15031528e-02, -7.51190484e-02,
        4.85178418e-02,  3.61067131e-02, -5.62809184e-02,  4.18828800e-02,
       -9.56438202e-03, -

In [120]:
X_test = get_word_vectors(test['description'])

In [122]:
rfc = RandomForestClassifier()

In [123]:
rfc.fit(X, train['ratingCategory'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [124]:
rfc.score(X, train['ratingCategory'])

1.0

### Make a Submission File

In [125]:
rfc.predict(X_test)

array([1, 1, 1, ..., 1, 1, 0])

In [127]:
test['ratingCategory'] = rfc.predict(X_test)

In [129]:
test[['id', 'ratingCategory']].to_csv('./data/testSolutionSubmission.csv', header=True, index=False)

In [ ]:
# OR USE THE FOLLOWING: 

submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    For our purposes, sentiment analysis is about the use of NLP to identify, extract, quantify, and study subjective information that can be gleaned from text and the context in which the text is found. Sentiment analysis can be used to identify how customers feel about products from the text in their reviews, and even how society may feel about politics and relgion, etc., based on their social media posts. It can be a powerful tool for marketing and advertising, especially if sentiment analysis is able to parse sarcastic from genuine sentiment. 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    Yes, document classification is different from sentiment analysis. Document classification, e.g. by way of k-nearest neighbors, is able to differentiate between words that belong in different categories, i.e. electronics vs clothing; however, it is unable to categorize based on positive or negative feelings (sentiment). For example, 'happy,' 'sad,' 'angry,' 'ecstatic,' would all be classified as feelings without bearing any positive or negative weights. 
    - How do create labeled sentiment data? Are those labels really sentiment?
    
    - What are common applications of sentiment analysis?
2. Research why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?